# Importação das bibliotecas

In [7]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
import pytz

import yfinance as yf
import MetaTrader5 as mt5

import kaleido
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Organizar datas de extração

In [13]:
hoje = datetime.now()
hoje

datetime.datetime(2024, 1, 22, 21, 43, 12, 341966)

In [14]:
hoje.weekday()

0

In [15]:
segunda = hoje - timedelta(days = hoje.weekday())
segunda

datetime.datetime(2024, 1, 22, 21, 43, 12, 341966)

In [16]:
segunda.date()

datetime.date(2024, 1, 22)

In [17]:
data_inicio = segunda.date().strftime('%Y-%m-%d')
data_final = (hoje.date()+ timedelta(days = 2)).strftime('%Y-%m-%d')
data_final

'2024-01-24'

In [18]:
def capa_post_weekly():

    semana_ano = segunda.date().strftime('%U')
    ano = segunda.date().strftime('%Y')

    print('Trading com Dados apresenta')
    print('Weekly Report')
    print(f'Semana {semana_ano} de {ano}')
    print(f'{segunda.date().strftime("%d-%m")} a {(segunda.date() + timedelta(days = 4)).strftime("%d-%m")}')

In [20]:
capa_post_weekly()

Trading com Dados apresenta
Weekly Report
Semana 03 de 2024
22-01 a 26-01


# Índices Internacionais

Extração e manipulação dos dados

In [21]:
dict_yf_world_indices = {
    'Bovespa (BVSP)': ['^BVSP','Brasil'],
    'S&P 500 (GSPC)': ['^GSPC', 'EUA'],
    'Dow Jones 30 (DJI)': ['^DJI', 'EUA'],
    'NASDAQ Composite (IXIC)': ['^IXIC', 'EUA'],
    'S&P/TSX Composite (GSPTSE)': ['^GSPTSE', 'Canadá'],
    'S&P/BMV IPC (MXX)': ['^MXX', 'México'],
    'S&P Merval (MERV)': ['^MERV', 'Argentina'],
    'Shanghai Composite (SSEC)': ['000001.SS', 'China'],
    'SZSE Component (SZI)': ['399001.SZ', 'China'],
    'Nikkei 225 (N225)': ['^N225', 'Japão'],
    'Hang Seng (HK50)': ['^HSI', 'Hong Kong'],
    'BSE Sensex 30 (BSESN)': ['^BSESN', 'Índia'],
    'EuroNext 100 (N100)': ['^N100', 'Zona Euro'],
    'DAX (DE30)': ['^GDAXI', 'Alemanha'],
    'FTSE 100 (UK100)': ['^FTSE', 'UK'],
    'CAC 40 (FCHI)': ['^FCHI', 'França'],
    'MOEX Russia (IMOEX)': ['IMOEX.ME', 'Rússia'],
    }

In [26]:
df_yf_world_indices = pd.DataFrame.from_dict(dict_yf_world_indices, orient='index', columns=['Ticker_YF', 'Country'])
df_yf_world_indices

,Ticker_YF,Country
Bovespa (BVSP),^BVSP,Brasil
S&P 500 (GSPC),^GSPC,EUA
Dow Jones 30 (DJI),^DJI,EUA
NASDAQ Composite (IXIC),^IXIC,EUA
S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá
S&P/BMV IPC (MXX),^MXX,México
S&P Merval (MERV),^MERV,Argentina
Shanghai Composite (SSEC),000001.SS,China
SZSE Component (SZI),399001.SZ,China
Nikkei 225 (N225),^N225,Japão


In [27]:
df_yf_world_indices = df_yf_world_indices.reset_index()
df_yf_world_indices = df_yf_world_indices.rename({'index':'Índice'}, axis=1)
df_yf_world_indices

,Índice,Ticker_YF,Country
0,Bovespa (BVSP),^BVSP,Brasil
1,S&P 500 (GSPC),^GSPC,EUA
2,Dow Jones 30 (DJI),^DJI,EUA
3,NASDAQ Composite (IXIC),^IXIC,EUA
4,S&P/TSX Composite (GSPTSE),^GSPTSE,Canadá
5,S&P/BMV IPC (MXX),^MXX,México
6,S&P Merval (MERV),^MERV,Argentina
7,Shanghai Composite (SSEC),000001.SS,China
8,SZSE Component (SZI),399001.SZ,China
9,Nikkei 225 (N225),^N225,Japão


In [28]:
ohlcv_indice = pd.DataFrame(yf.download(df_yf_world_indices.Ticker_YF.values[0], start=data_inicio, end=data_final, progress=False, interval="1d"))
ohlcv_indice

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-22 00:00:00-03:00,127636.320312,127842.59375,125875.648438,126601.546875,126601.546875,0


In [29]:
ohlcv_indice = ohlcv_indice.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
ohlcv_indice["Ticker_YF"] = df_yf_world_indices.Ticker_YF.values[0]
ohlcv_indice

,Open,High,Low,Close,Ticker_YF
Date,,,,,
2024-01-28 00:00:00-03:00,127636.320312,127842.59375,125875.648438,126601.546875,^BVSP


In [30]:
ohlcv_indice["Resultado_%"] = (ohlcv_indice.Close/ohlcv_indice.Open - 1) * 100
ohlcv_indice

,Open,High,Low,Close,Ticker_YF,Resultado_%
Date,,,,,,
2024-01-28 00:00:00-03:00,127636.320312,127842.59375,125875.648438,126601.546875,^BVSP,-0.81072


In [33]:
world_ohlcv = pd.DataFrame()

for i in df_yf_world_indices.Ticker_YF:

    try:
        ohlcv_indice = pd.DataFrame(yf.download(i, start=data_inicio, end=data_final, progress=False, interval="1d"))
        ohlcv_indice = ohlcv_indice.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
        ohlcv_indice["Ticker_YF"] = i
        ohlcv_indice["Resultado_%"] = (ohlcv_indice.Close/ohlcv_indice.Open - 1) * 100
        world_ohlcv = pd.concat([world_ohlcv, ohlcv_indice], axis=0)

    except:
        pass


1 Failed download:
- ^MERV: No data found for this date range, symbol may be delisted

1 Failed download:
- ^BSESN: No data found for this date range, symbol may be delisted

1 Failed download:
- IMOEX.ME: No data found for this date range, symbol may be delisted
